# Running K-fold CV

This notebook details how to run k-fold CV for the saved models. Here, 10 folds are used.
The contents here are the same as in `KfoldCV.R`.

In [2]:

library("rstanarm")

# First, load the data and premade folds:
data <- readRDS("strain_surv_data/data.RData")
data_sites <- readRDS("strain_surv_data/data_sites.RData")
treatments <- readRDS("strain_surv_data/treatments.RData")

# Fix 
data <- na.omit(data[,c("strain", "ARM", "host_id", "ST", "y", "t0", "delta_t", treatments)]) # omit rows (of vars that are used in the model) with NAs
data_sites <- na.omit(data_sites[,c("strain", "ARM", "host_id", "ST", "site", "y", "t0", "delta_t", treatments)])

get_new_folds <- T # Do you wish to create new folds using get_folds function?

print("Loaded data.")

get_folds <- function(data, seed = 02122021){
  #' Create new folds using loo::kfold_split_stratified
    
  set.seed(seed)
  
  folds_strain_d <- loo::kfold_split_stratified(K = 10, x = data[which(data$ARM == 1),"strain"])
  folds_strain_e <- loo::kfold_split_stratified(K = 10, x = data[which(data$ARM == 0),"strain"])
  
  saveRDS(folds_strain_d, file = "folds_strain_d.RData")
  saveRDS(folds_strain_e, file = "folds_strain_e.RData")
  
  print("Folds created and saved.")
}

if (get_new_folds){
  get_folds(data)
}


folds_strain_d <- readRDS("folds_strain_d.RData")
folds_strain_e <- readRDS("folds_strain_e.RData")

[1] "Loaded data."


Warning message in bins[xids] <- rep(1:K, ceiling(N/K))[1:N]:
“number of items to replace is not a multiple of replacement length”
Warning message in bins[xids] <- rep(1:K, ceiling(N/K))[1:N]:
“number of items to replace is not a multiple of replacement length”


[1] "Folds created and saved."


In [8]:
options(mc.cores = parallel::detectCores())

# Read raneff_formula, savefile and arm from command line (saved to txt files)
args = c("raneff_formula.txt", "res/resD/10foldCV_fixed.Rdata", 1)#commandArgs(trailingOnly=TRUE)
print(args)
print(length(args))

raneff <- "fixed"
print(raneff)

raneff_formula <- gsub(",", "",toString(paste(paste('+ (1', raneff, sep = '|'), ')', sep = '')))

if(raneff == "fixed"){
  raneff_formula <- ''
}

savefile <- args[2]
arm <- as.numeric(args[3]) # decolonization or education

print(args[2])
print(args[3])

data <- data[which(data$ARM == arm),]

if (arm == 1){
    folds_strain <- folds_strain_d
}

if(arm == 0){
    folds_strain <- folds_strain_e    
}

pooled_formula <- paste(paste('Surv(time = t0, time2 = delta_t, event = y, type = "interval") ~', gsub(',', ' + ', toString(treatments)), sep = ''), raneff_formula, sep = '')


[1] "raneff_formula.txt"            "res/resD/10foldCV_fixed.Rdata"
[3] "1"                            
[1] 3
[1] "fixed"
[1] "res/resD/10foldCV_fixed.Rdata"
[1] "1"


In [9]:
print(pooled_formula)

# Run model 2 (all antibiotics) without random effects (faster to run locally). 
# Use higher adapt_delta and iter with random effects to avoid divergent transitions and ESS tail problems
# (see rstanarm documentation for details)

print(paste("Generating model: ", raneff_formula))
print(paste("ARM", arm))

mod <- stan_surv(data = na.omit(data[which(data$ARM == arm),]), formula = as.formula(pooled_formula),
                 basehaz = "exp", refresh = 0,
                 prior_intercept = normal(0,20), prior = normal(0,2.5),
            iter = 7500, adapt_delta = 0.999) #iter = 10000, adapt_delta = 0.99)


[1] "Surv(time = t0, time2 = delta_t, event = y, type = \"interval\") ~Ciprofloxacin +  Clindamycin +  Erythromycin +  Gentamicin +  Mupirocin +  Rifampicin +  Tetracycline +  Trimethoprim +  Chlorhexidine"
[1] "Generating model:  "
[1] "ARM 1"


In [10]:
# Kfold CV trick for stan_surv:
# Courtesy of John Baums, https://github.com/stan-dev/rstanarm/issues/473
f <- eval(parse(
  text=sub(
    'fit_k_call$subset <- eval(fit_k_call$subset)', 
    'fit_k_call$subset <- if(is.stansurv(x)) NULL else eval(fit_k_call$subset)', 
    deparse(rstanarm:::kfold.stanreg), fixed=TRUE)
), envir=environment(rstanarm:::kfold.stanreg))

assignInNamespace('kfold.stanreg', value=f, ns='rstanarm')

# Run kfold:
print("Running 10-fold CV...")
kfold_data <- kfold(mod, K=10, folds = folds_strain)
saveRDS(kfold_data, file = savefile)
print("Done.")


[1] "Running 10-fold CV..."


Fitting K = 10 models distributed over 8 cores



[1] "Done."
